In [4]:
#빅데이터분석기사 6회 실기 작업 1유형. 연평균 범죄율이 가장 많이 증가한 경찰서 찾기
#데이터셋
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

def datarange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = datetime(2022,1,31)
end_date = datetime(2023,12,31)

dates = list(datarange(start_date,end_date))
police_station = [f'경찰서{i}' for i in range(1,21)]

station_names = []
date_list = []
violence_counts = []
theft_counts = []
traffic_violation_counts = []
fraud_counts = []
other_counts = []

for station in police_station:
    for date in dates:
        station_names.append(station)
        date_list.append(date.strftime('%Y-%m-%d'))
        violence_counts.append(np.random.randint(0, 50))
        theft_counts.append(np.random.randint(0, 50))
        traffic_violation_counts.append(np.random.randint(0, 50))
        fraud_counts.append(np.random.randint(0, 50))
        other_counts.append(np.random.randint(0, 50))
        
new_data = pd.DataFrame({
    '경찰서명': station_names,
    '날짜' : date_list,
    '폭력건수' : violence_counts,
    '절도건수' : theft_counts,
    '교통법규위반건수' : traffic_violation_counts,
    '사기건수' : fraud_counts,
    '기타건수' : other_counts
})

new_data.head()

,경찰서명,날짜,폭력건수,절도건수,교통법규위반건수,사기건수,기타건수
0,경찰서1,2022-01-31,24,35,25,34,6
1,경찰서1,2022-02-01,34,30,17,24,41
2,경찰서1,2022-02-02,10,14,49,13,47
3,경찰서1,2022-02-03,45,29,17,12,28
4,경찰서1,2022-02-04,26,17,31,21,9


In [22]:
#1. 날짜 열을 datetime 형식으로 변환
new_data['날짜'] = pd.to_datetime(new_data['날짜'])
new_data.head()
new_data.info()

#2. 일자별 전체 범죄 건수 데이터 생성
new_data['총 범죄건수'] = new_data.loc[:,'폭력건수':'기타건수'].sum(axis=1) #슬라이싱활용, axis=1은 가로방향 행단위의 합계를 구하기 위해/ axis=0은 열기준으로 합계를 구한다
new_data.head()

#3. 2022년과 2023년으로 데이터 셋을 변경
data_2022 = new_data.loc[new_data['날짜'].dt.year==2022,:]
data_2023 = new_data.loc[new_data['날짜'].dt.year==2023,:]

#4. 각 경찰서 별로 2022년과 2023년의 연간 평균 범죄 건수를 계산하시오.
avg_crime_data_2022 = data_2022.groupby('경찰서명')['총 범죄건수'].mean().reset_index()
avg_crime_data_2023 = data_2023.groupby('경찰서명')['총 범죄건수'].mean().reset_index()

avg_crime_data_2022.head()
avg_crime_data_2023.head()

#5. 두 연도의 평균을 병합하시오.
crime_comparison = avg_crime_data_2022.merge(avg_crime_data_2023, on='경찰서명', suffixes=('_2022', '_2023'))
#on='경찰서명' 병합 기준이 되는 공통 열(즉 경찰서명이 같은 행끼리 합쳐진다.)
#suffixes=('_2022','_2023')은 겹치는 열 이름들에 붙이는 접미사(예: 범죄건수_2022, 범죄건수_2023)
crime_comparison

#6. 두 연도의 차이를 계산하시오.
crime_comparison['증가율'] = crime_comparison['총 범죄건수_2023']-crime_comparison['총 범죄건수_2022']
crime_comparison

#7. 평균 범죄율이 가장 많이 증가한 경찰서를 찾기
max_increase_station = crime_comparison.loc[crime_comparison['증가율'].idxmax(),'경찰서명']
max_increase_station

#8. 2022년 5월에 해당 경찰서의 총 사기건수를 계산하시오.
result =data_2022.loc[ (data_2022['경찰서명']==max_increase_station) & (data_2022['날짜'].dt.month==5)]
result['사기건수'].sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13980 entries, 0 to 13979
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   경찰서명      13980 non-null  object        
 1   날짜        13980 non-null  datetime64[ns]
 2   폭력건수      13980 non-null  int64         
 3   절도건수      13980 non-null  int64         
 4   교통법규위반건수  13980 non-null  int64         
 5   사기건수      13980 non-null  int64         
 6   기타건수      13980 non-null  int64         
 7   총 범죄건수    13980 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 873.9+ KB


819